<a href="https://colab.research.google.com/github/tianhaoz95/ultron/blob/dev%2Ftry-atari-game/notebooks/prototype_atari_with_actor_critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay tqdm > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg x11-utils > /dev/null 2>&1

In [2]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install gputil > /dev/null 2>&1
!pip install pyglet==1.2.4 > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (46.1.3)


In [40]:
import tensorflow as tf
import GPUtil
print("Tensorflow version " + tf.__version__)
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
  try:
    print('ERROR: Not connected to a TPU runtime!')
    GPUs = GPUtil.getGPUs()
    print('GPU count: ' + str(len(GPUs)))
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
      print('and then re-execute this cell.')
    else:
      print(gpu_info)
  except:
    print('Using CPU')

Tensorflow version 2.2.0-rc4
ERROR: Not connected to a TPU runtime!
GPU count: 1
Fri May  8 19:50:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    32W / 250W |   8819MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Proce

In [67]:
import gym
import threading
import math
import glob
import io
import base64
import multiprocessing
import matplotlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from gym.wrappers import Monitor
from tensorflow import keras
from os import path
from time import sleep
from tqdm.notebook import tqdm
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [68]:
print('using tensorflow', tf.__version__)

using tensorflow 2.2.0-rc4


In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

def wrap_env(env):
  wrapped_env = Monitor(env, './video', force=True)
  return wrapped_env

In [0]:
class PipelineArgs():
  def __init__(self,
               game_name='Acrobot-v1',
               mode='train',
               max_eps=100,
               update_freq=20,
               gamma=0.99,
               model_path='.',
               load_model=False,
               lr=0.001):
    self.game_name = game_name
    self.mode = mode
    self.gamma = gamma
    self.update_freq = update_freq
    self.lr = lr
    self.max_eps = max_eps
    self.load_model = load_model
    self.model_path = model_path

In [0]:
class History():
  def __init__(self, args):
    self.history = pd.DataFrame()
    self.args = args
  
  def record(self, eps_reward):
    self.eps_rewards.append(eps_reward)

  def save(self):
    self.history['eps_rewards'] = self.eps_rewards
    self.history.to_csv(path.join(self.args.model_path, 'history.csv'))
  
  def load(self):
    if path.exists(
        path.join(
            self.args.model_path, 'history.csv')) and self.args.load_model:
      self.history = pd.DataFrame()
      self.history.read_csv(path.join(self.args.model_path, 'history.csv'))

In [0]:
class Memory():
  def __init__(self):
    self.states = []
    self.actions = []
    self.rewards = []
  
  def store(self, state, action, reward):
    self.states.append(state)
    self.actions.append(action)
    self.rewards.append(reward)

  def clear(self):
    self.states = []
    self.actions = []
    self.rewards = []

In [0]:
class Worker(threading.Thread):
  # The cumulative episode played
  global_eps = 0
  best_score = 0
  lock = threading.Lock()

  def __init__(self,
               args,
               action_size,
               state_size,
               global_model,
               opt,
               pbar):
    super(Worker, self).__init__()
    self.args = args
    self.game_name = self.args.game_name
    self.env = gym.make(self.game_name)
    self.action_size = action_size
    self.state_size = state_size
    self.local_model = ActorCriticModel(self.state_size, self.action_size)
    self.eps_loss = 0
    self.opt = opt
    self.global_model = global_model
    self.pbar = pbar

  def compute_loss(self, done, new_state, memory):
    if done:
      reward_sum = 0
    else:
      reward_sum = self.local_model(
          tf.convert_to_tensor(
              new_state[None, :],
              dtype=tf.float32))[-1].numpy()[0]
    discounted_rewards = []
    for reward in memory.rewards[::-1]:
      reward_sum = reward + self.args.gamma * reward_sum
      discounted_rewards.append(reward_sum)
    discounted_rewards.reverse()
    past_states = np.array(memory.states)
    logits, values = self.local_model(
        tf.convert_to_tensor(past_states, dtype=tf.float32))
    advantage = tf.convert_to_tensor(
        np.array(discounted_rewards)[:, None], dtype=tf.float32) - values
    # Calculate the loss for value function which mean how off is our
    # predicted value from the true value estimated from the discounted
    # reward.
    value_loss = advantage ** 2
    # Calculate the policy loss
    policy = tf.nn.softmax(logits)
    entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=policy, logits=logits)
    policy_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=memory.actions, logits=logits)
    policy_loss *= tf.stop_gradient(advantage)
    policy_loss -= 0.01 * entropy
    # Combine the value and policy loss to be a single trainable
    total_loss = tf.reduce_mean((0.5 * value_loss + policy_loss))
    return total_loss

  def compute_and_apply_gradient(self, done, new_state, mem):
    with tf.GradientTape() as tape:
      total_loss = self.compute_loss(done, new_state, mem)
    self.eps_loss += total_loss
    grads = tape.gradient(
        total_loss, self.local_model.trainable_weights)
    self.opt.apply_gradients(zip(
        grads, self.global_model.trainable_weights))
    self.local_model.set_weights(self.global_model.get_weights())

  def run(self):
    # Prepare variables
    mem = Memory()
    total_steps = 1
    # Check the maximum episode of learning is reached
    while Worker.global_eps < self.args.max_eps:
      current_state = self.env.reset()
      mem.clear()
      done = False
      ep_reward = 0
      self.eps_loss = 0
      time_count = 0
      ep_steps = 0
      # Check if the game is over
      while not done:
        # Get the policy and play the game
        logits, _ = self.local_model(
            tf.convert_to_tensor(np.array([current_state]), dtype=tf.float32))
        probs = tf.nn.softmax(logits)
        action = np.random.choice(self.action_size, p=probs.numpy()[0])
        new_state, reward, done, info = self.env.step(action)
        ep_reward += reward
        mem.store(current_state, action, reward)
        # If the explore time limit has been reached or
        # the game is over, then update the models
        if time_count >= self.args.update_freq or done:
          self.compute_and_apply_gradient(done, new_state, mem)
          mem.clear()
          time_count = 0
          if done:
            with Worker.lock:
              if Worker.global_eps == 0:
                Worker.best_score = ep_reward
              if ep_reward > Worker.best_score:
                self.global_model.save_weights(
                    path.join(self.args.model_path, 'best.h5'))
                Worker.best_score = ep_reward
              else:
                self.global_model.save_weights(
                    path.join(self.args.model_path, 'backup.h5'))
              Worker.global_eps += 1
              self.pbar.set_description(
                  f'step: {ep_steps}, reward {ep_reward}/{Worker.best_score}')
              self.pbar.update(1)
        ep_steps += 1
        time_count += 1
        current_state = new_state
        total_steps += 1

In [0]:
class ActorCriticModel(keras.Model):
  def __init__(self, state_size, action_size):
    super(ActorCriticModel, self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.conv_0 = keras.layers.Conv2D(
        filters=4,
        activation='relu',
        kernel_size=8,
        kernel_initializer='random_uniform',
        input_shape=(None, 210, 160, 3))
    self.conv_1 = keras.layers.Conv2D(
        filters=4,
        activation='relu',
        kernel_initializer='random_uniform',
        kernel_size=8)
    self.flatten  =keras.layers.Flatten()
    self.policy_dense_0 = keras.layers.Dense(
        64, kernel_initializer='random_uniform', activation='relu')
    self.policy_dense_1 = keras.layers.Dense(
        32, kernel_initializer='random_uniform', activation='relu')
    self.policy_logits = keras.layers.Dense(self.action_size, activation='relu')
    self.value_dense = keras.layers.Dense(
        128, kernel_initializer='random_uniform', activation='relu')
    self.value = keras.layers.Dense(1)
  
  def call(self, inputs):
    x = self.conv_0(inputs)
    x = self.conv_1(x)
    x = self.flatten(x)
    p = self.policy_dense_0(x)
    p = self.policy_dense_1(p)
    logits = self.policy_logits(p)
    v = self.value_dense(x)
    values = self.value(v)
    return logits, values

In [0]:
class BaselineModel():
  def __init__(self):
    print('not implemented')

In [0]:
class MasterAgent():
  def __init__(self, args):
    self.args = args
    self.game_name = self.args.game_name
    env = gym.make(self.game_name)
    self.action_size = env.action_space.n
    self.state_size = env.observation_space.shape[0]
    self.opt = tf.optimizers.Adam(self.args.lr)
    print('state_size: ', self.state_size)
    print('action_size: ', self.action_size)
    self.global_model = ActorCriticModel(self.state_size, self.action_size)
    self.global_model(
      tf.convert_to_tensor(
      np.random.random((5, 210, 160, 3)),
      dtype=tf.float32))
    if path.exists(
        path.join(self.args.model_path, 'best.h5')) and self.args.load_model:
      self.global_model.load_weights(
          path.join(self.args.model_path, 'best.h5'))
    self.pbar = tqdm(total=self.args.max_eps)
  
  def train_sync(self):
    worker = Worker(self.args, self.action_size, self.state_size,
                    self.global_model, self.opt, self.pbar)
    worker.run()
    self.pbar.close()
  
  def train_async(self):
    Worker.global_eps = 0;
    workers = [
      Worker(self.args, self.action_size, self.state_size,
             self.global_model, self.opt, self.pbar)
      for i in range(multiprocessing.cpu_count())
    ]
    print(f'using {len(workers)} worker in parallel')
    for i, worker in enumerate(workers):
      worker.start()
    [w.join() for w in workers]
    self.pbar.close()
    self.play()
  
  def play(self):
    self.pbar.close()
    env = wrap_env(gym.make(self.args.game_name))
    state = env.reset()
    model = self.global_model
    if path.exists(
        path.join(self.args.model_path, 'best.h5')) and self.args.load_model:
      model.load_weights(
          path.join(self.args.model_path, 'best.h5'))
    done = False
    while not done:
      state_input = tf.convert_to_tensor(state[None, :], dtype=tf.float32)
      logit, _ = model(state_input)
      policy = tf.nn.softmax(logit)
      action = np.argmax(policy)
      state, reward, done, _ = env.step(action)
    env.close()
    show_video()

In [0]:
def gym_sanity_check(args):
  print('starting gym environment sanity check')
  env = wrap_env(gym.make(args.game_name))
  inital_observation = env.reset()
  sample_action = env.action_space.sample()
  print('observation shape: ', np.array(inital_observation).shape)
  print('sample action: ', sample_action)
  for _ in range(20):
    observation = env.reset()
    done = False
    while not done:
        action = env.action_space.sample()
        observation, _, done, _ = env.step(action)
  env.close()
  show_video()

In [0]:
def entry_point(args):
  if args.mode == 'sanity':
    gym_sanity_check(args)
  if args.mode == 'play':
    master = MasterAgent(args)
    master.play()
  if args.mode == 'train_sync':
    master = MasterAgent(args)
    master.train_sync()
  if args.mode == 'train_async':
    master = MasterAgent(args)
    master.train_async()
  print('Hello World')

In [80]:
entry_point(
    PipelineArgs(
        game_name='Berzerk-v0',
        update_freq=64,
        max_eps=2000,
        load_model=True,
        mode='play'))

state_size:  210
action_size:  18


Hello World
